## Klasifikasi Text - Tugas 3 Pengenalan Pola

## Baskara - 16/398499/PA/17460

### Import Library

In [1]:
import pandas as pd
import numpy as np
import nltk
import math
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

### Load Data Training

In [2]:
df = pd.read_csv('training.csv', names=["text","penyakit"])  
df.head()

,text,penyakit
0,Muntah berak kencing berak,kanker
1,demam darah,pilek
2,Muntah darah pusing kencing,sariawan
3,berdarah kencing,kanker


### Tokenization & Stemming Data Training

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

for index, row in df.iterrows():
    # Stemming
    stemmed = stemmer.stem(row[0])
    #Tokenization
    tokens = nltk.word_tokenize(row[0])
    #Case Folding
    words = [w.lower() for w in tokens]
    df.at[index, 'text'] = words

In [4]:
df.head()

,text,penyakit
0,"[muntah, berak, kencing, berak]",kanker
1,"[demam, darah]",pilek
2,"[muntah, darah, pusing, kencing]",sariawan
3,"[berdarah, kencing]",kanker


### Membuat Kolom Untuk Setiap Kata

In [5]:
columnlist = []
for index, row in df.iterrows():
    columnlist = np.concatenate((columnlist, row[0]))
columnlist = np.unique(columnlist)

In [6]:
for index in range(len(columnlist)):
    df[str(columnlist[index])] = 0

In [7]:
df.head()

,text,penyakit,berak,berdarah,darah,demam,kencing,muntah,pusing
0,"[muntah, berak, kencing, berak]",kanker,0,0,0,0,0,0,0
1,"[demam, darah]",pilek,0,0,0,0,0,0,0
2,"[muntah, darah, pusing, kencing]",sariawan,0,0,0,0,0,0,0
3,"[berdarah, kencing]",kanker,0,0,0,0,0,0,0


### Menghitung Jumlah Frekuensi Setiap Kata

In [8]:
for index, row in df.iterrows():
    for columnindex in range(len(columnlist)):
        df.at[index, columnlist[columnindex]] = row[0].count(columnlist[columnindex])

In [9]:
df.head()

,text,penyakit,berak,berdarah,darah,demam,kencing,muntah,pusing
0,"[muntah, berak, kencing, berak]",kanker,2,0,0,0,1,1,0
1,"[demam, darah]",pilek,0,0,1,1,0,0,0
2,"[muntah, darah, pusing, kencing]",sariawan,0,0,1,0,1,1,1
3,"[berdarah, kencing]",kanker,0,1,0,0,1,0,0


### Load Data Test

In [10]:
test = pd.read_csv('test.csv', names=["text"])
test['prediksi'] = ''
test['jarak'] = 0.0

### Preprocessing dan Penghitungan Jumlah Frekuensi Setiap Kata

In [11]:
for index, row in test.iterrows():
    # Stemming
    stemmed = stemmer.stem(row[0])
    #Tokenization
    tokens = nltk.word_tokenize(row[0])
    #Case Folding
    words = [w.lower() for w in tokens]
    test.at[index, 'text'] = words
for index in range(len(columnlist)):
    test[str(columnlist[index])] = 0
for index, row in test.iterrows():
    for columnindex in range(len(columnlist)):
        test.at[index, columnlist[columnindex]] = row[0].count(columnlist[columnindex])

### Penghitungan Jarak (Menggunakan Eucledian)

In [12]:
for test_index, test_row in test.iterrows():
    distance = []
    for train_index, train_row in df.iterrows():
        temp = 0
        for columnindex in range(len(columnlist)):
            temp = temp + (test_row[3+columnindex] - train_row[2+columnindex])**2
        distance += [math.sqrt(temp)]
    test.at[test_index, 'jarak'] = (np.min(distance))
    test.at[test_index, 'prediksi'] = str(df['penyakit'][np.argmin(distance)])

### Hasil Prediksi

In [13]:
test.head(10)

,text,prediksi,jarak,berak,berdarah,darah,demam,kencing,muntah,pusing
0,"[muntah, berak, kencing, berak]",kanker,0.0,2,0,0,0,1,1,0
1,"[demam, darah, kismis]",pilek,0.0,0,0,1,1,0,0,0
2,"[muntah, darah, pusing, kencing]",sariawan,0.0,0,0,1,0,1,1,1
3,"[berdarah, kencing]",kanker,0.0,0,1,0,0,1,0,0
4,"[demam, doang]",pilek,1.0,0,0,0,1,0,0,0
5,"[muntah, muntah]",sariawan,2.0,0,0,0,0,0,2,0
6,"[demam, demaman]",pilek,1.0,0,0,0,1,0,0,0
